In [61]:
import scipy.io
import numpy as np
import networkx as nx
import os
import pandas as pd
from sklearn.cluster import SpectralClustering
# import community  # Install using: pip install python-louvain
import matplotlib.pyplot as plt

In [51]:
##########################################################################################
#                                    ALL FUNCTIONS
##########################################################################################

def get_graph_from_file(file, flag):
    if flag:
        print(f'Reading the network file: {file}\n')
    
    return nx.read_weighted_edgelist(file, nodetype=str)


def graph_properties(G, flag, file):
    '''
    Parameters:
        G (graph)
        flag (int): flag=1 to print graph stats, 0 else
    '''
    
    n = len(G.nodes())
    m = len(G.edges())
    C = nx.transitivity(G)
    degrees = nx.degree(G)
    kis = [k for _,k in degrees]
    kstd = np.std(kis)
    kmean = np.mean(kis)
    max_edges = n * (n - 1) / 2
    edge_density = m / max_edges
    if nx.is_connected(G):
        average_shortest_path_length = nx.average_shortest_path_length(G, weight='weight')
    else :
        average_shortest_path_length = 0
    
    betweenness = nx.betweenness_centrality(G, weight='weight')
    non_zero_betweenness = [value for value in betweenness.values() if value != 0]
    avg_betweenness_centrality = np.mean(non_zero_betweenness)

    total_weight = sum(data['weight'] for _, _, data in G.edges(data=True))
    average_edge_weight = total_weight / G.number_of_edges()

    if flag:
        print("Number of nodes (n):", n)
        print("Number of edges (m):", m)
        print("Clustering Coefficient (C):", C)
        print("Mean Degree (kmean):", kmean)
        print("Degree STD (kstd):", kstd)
        print("Edge density (D):", edge_density)
        print("Average shortest path length: ", average_shortest_path_length)
        print("Average betweeness centrality: ", avg_betweenness_centrality)
        print("Average edge weight: ", average_edge_weight)
        print("\n")
        
    return [C,kstd,kmean,edge_density,average_shortest_path_length,
            avg_betweenness_centrality,average_edge_weight, file]


def get_all_vectors(directory, flag):
    '''
    Parameters:
        directory path (string)
        flag (int): flag=1 to print graph stats, 0 else
    '''
    files = os.listdir(directory)  # Slicing should be done after calling listdir()
    all_graphs = []
    all_graphs_properties = []
    
    flag_count = 0
    files = list(set(files))
    
    for file in files:
        file_pathway = directory+file
        if flag_count >= 5:
            flag = 0
            break
            
        G = get_graph_from_file(file_pathway, flag) 
        p = graph_properties(G, flag, file)
        all_graphs.append(G)
        all_graphs_properties.append(p)
        flag_count += 1
        
    return all_graphs_properties

def lower_distance_matrix(df):
    """
    Calculate the lower half of the distance matrix for vectors stored in a DataFrame using Euclidean
    
    Args:
        df (pandas.DataFrame): DataFrame containing vectors as rows.
        
    Returns:
        pandas.DataFrame: Lower half of the distance matrix.
    """
    
    num_rows = df.shape[0]
    distance_matrix = np.zeros((num_rows, num_rows))
    
    # Calculate pairwise distances
    for i in range(num_rows):
        for j in range(i):
            distance_matrix[i, j] = euclidean_distance(df.iloc[i], df.iloc[j])
    
    # Convert to DataFrame with filename as index
    distance_df = pd.DataFrame(distance_matrix, index=df.index, columns=df.index)
    
    return distance_df


In [89]:
# A Quick Implementation of UPGMA (Unweighted Pair Group Method with Arithmetic Mean)

# lowest_cell:
#   Locates the smallest cell in the table
def lowest_cell(table):
    # Set default to infinity
    min_cell = float("inf")
    x, y = -1, -1

    # Go through every cell, looking for the lowest
    for i in range(len(table)):
        for j in range(len(table[i])):
            if table[i][j] < min_cell:
                min_cell = table[i][j]
                x, y = i, j

    # Return the x, y co-ordinate of cell
    return x, y


# join_labels:
#   Combines two labels in a list of labels
def join_labels(labels, a, b):
    # Swap if the indices are not ordered
    if b < a:
        a, b = b, a

    # Join the labels in the first index
    labels[a] = "(" + labels[a] + "," + labels[b] + ")"

    # Remove the (now redundant) label in the second index
    del labels[b]


# join_table:
#   Joins the entries of a table on the cell (a, b) by averaging their data entries
def join_table(table, a, b):
    # Swap if the indices are not ordered
    if b < a:
        a, b = b, a

    # For the lower index, reconstruct the entire row (A, i), where i < A
    row = []
    for i in range(0, a):
        row.append((table[a][i] + table[b][i])/2)
    table[a] = row
    
    # Then, reconstruct the entire column (i, A), where i > A
    #   Note: Since the matrix is lower triangular, row b only contains values for indices < b
    for i in range(a+1, b):
        table[i][a] = (table[i][a]+table[b][i])/2
        
    #   We get the rest of the values from row i
    for i in range(b+1, len(table)):
        table[i][a] = (table[i][a]+table[i][b])/2
        # Remove the (now redundant) second index column entry
        del table[i][b]

    # Remove the (now redundant) second index row
    del table[b]


# UPGMA:
#   Runs the UPGMA algorithm on a labelled table
def UPGMA(table, labels):
    # Until all labels have been joined...
    while len(labels) > 1:
        # Locate lowest cell in the table
        x, y = lowest_cell(table)

        # Join the table on the cell co-ordinates
        join_table(table, x, y)

        # Update the labels accordingly
        join_labels(labels, x, y)

    # Return the final label
    return labels[0]


## A test using an example calculation from http://www.nmsr.org/upgma.htm

# alpha_labels:
#   Makes labels from a starting letter to an ending letter
def alpha_labels(start, end):
    labels = []
    for i in range(ord(start), ord(end)+1):
        labels.append(i)
    return labels

def lower_distance_to_M_data(lower_distance_df):
    """
    Convert lower distance DataFrame to M_data format for UPGMA.
    
    Args:
        lower_distance_df (pandas.DataFrame): Lower half of the distance matrix.
        
    Returns:
        list: M_data list for UPGMA.
    """
    M_data = []
    num_rows = lower_distance_df.shape[0]
    for i in range(num_rows):
        row_data = []
        for j in range(i):
            row_data.append(lower_distance_df.iloc[i, j])
        M_data.append(row_data)
    return M_data

In [35]:
####### Helpers for clustering #######

# def louvain_clustering(G, flag):
#     partition = community.best_partition(G)
#     num_clusters = len(set(partition.values())) # Count the number of unique community IDs\

#     if flag:
#         # Draw the graph with nodes colored by community
#         pos = nx.spring_layout(G)  # Positions for all nodes
#         plt.figure(figsize=(8, 6))

#         # Draw nodes, coloring them by community
#         node_colors = [partition[n] for n in G.nodes()]
#         nx.draw(G, pos, node_color=node_colors, cmap=plt.cm.tab10, node_size=20)

#         plt.title("Graph with Nodes Colored by Community (Louvain Method)")
#         plt.show()
        
#     return partition, num_clusters

In [30]:
# G = nx.karate_club_graph

In [31]:
# fname1 = 'fungal_networks_data/Conductance_Text_Files/Sc_M_I_U_N_42d_1.txt'
# G = get_graph_from_file(fname1, fname1)

Reading the network file: fungal_networks_data/Conductance_Text_Files/Sc_M_I_U_N_42d_1.txt



In [32]:
#########    TEST HELPER FUNCTIONS FOR ONE FILE    #########

# partition, num_clusters = louvain_clustering(G, 1)
graph_properties(G, 1, fname1)
# print(f'Number of clusters = {num_clusters}')

Number of nodes (n): 536
Number of edges (m): 689
Clustering Coefficient (C): 0.08066581306017925
Mean Degree (kmean): 2.5708955223880596
Degree STD (kstd): 1.3378153625625286
Edge density (D): 0.0048054121913795505
Average shortest path length:  0.9921940403124676
Average betweeness centrality:  0.03839205877954172
Average edge weight:  0.11330391872278674




[0.08066581306017925,
 1.3378153625625286,
 2.5708955223880596,
 0.0048054121913795505,
 0.9921940403124676,
 0.03839205877954172,
 0.11330391872278674,
 'fungal_networks_data/Conductance_Text_Files/Sc_M_I_U_N_42d_1.txt']

In [90]:
##################     GET ALL GRAPHS AND PROPERTIES     ##################
flag = 0
directory = 'fungal_networks_data/Conductance_Text_Files/'
gmatrix = get_all_vectors(directory, flag)
column_names = ['C', 'kstd', 'kmean', 'edge_density','average_shortest_path_length','avg_betweenness_centrality','average_edge_weight','file']
df = pd.DataFrame(gmatrix, columns=column_names)
df.set_index('file', inplace=True)

print(df)

                                  C      kstd     kmean  edge_density  \
file                                                                    
Pv_L_I+4xR_Fc_N_78d_1.txt  0.081688  1.044973  2.561525      0.004447   
Pv_M_I_U_N_19d_2.txt       0.053070  1.234588  2.247748      0.005074   
Pv_M_5xI_U_N_22d_3.txt     0.083542  1.122883  2.555283      0.002094   
Pv_M_I_U_N_43d_3.txt       0.075055  1.175252  2.386540      0.002061   
Pp_M_Tokyo_U_N_26h_7.txt   0.201342  0.855373  3.197917      0.016743   

                           average_shortest_path_length  \
file                                                      
Pv_L_I+4xR_Fc_N_78d_1.txt                      0.000000   
Pv_M_I_U_N_19d_2.txt                           0.005051   
Pv_M_5xI_U_N_22d_3.txt                         0.000000   
Pv_M_I_U_N_43d_3.txt                           0.005891   
Pp_M_Tokyo_U_N_26h_7.txt                   37208.881893   

                           avg_betweenness_centrality  average_edge_weight

In [91]:
##################     GET DISTANCE MATRIX     ##################
lower_distance_df = lower_distance_matrix(df)
print(lower_distance_df.to_string(float_format=lambda x: "%.2f" % x))


##################     FINISHED     ##################
print("Fin")

file                       Pv_L_I+4xR_Fc_N_78d_1.txt  Pv_M_I_U_N_19d_2.txt  Pv_M_5xI_U_N_22d_3.txt  Pv_M_I_U_N_43d_3.txt  Pp_M_Tokyo_U_N_26h_7.txt
file                                                                                                                                              
Pv_L_I+4xR_Fc_N_78d_1.txt                       0.00                  0.00                    0.00                  0.00                      0.00
Pv_M_I_U_N_19d_2.txt                            0.37                  0.00                    0.00                  0.00                      0.00
Pv_M_5xI_U_N_22d_3.txt                          0.08                  0.33                    0.00                  0.00                      0.00
Pv_M_I_U_N_43d_3.txt                            0.22                  0.15                    0.18                  0.00                      0.00
Pp_M_Tokyo_U_N_26h_7.txt                    38870.68              38870.68                38870.68              38870.

In [92]:
##################     SMALL TEST ON THING       ##################

M_labels = lower_distance_df.index.tolist()
M = lower_distance_to_M_data(lower_distance_df)
UPGMA(M, M_labels)

'(((Pv_L_I+4xR_Fc_N_78d_1.txt,Pv_M_5xI_U_N_22d_3.txt),(Pv_M_I_U_N_19d_2.txt,Pv_M_I_U_N_43d_3.txt)),Pp_M_Tokyo_U_N_26h_7.txt)'

In [50]:
##################     TEST DIST BTWEEN TWO VECS     ##################

vec1 = df.loc['Pp_M_Tokyo_U_N_26h_7.txt']
vec2 = df.loc['Pv_L_I+4xR_Fc_N_78d_1.txt']
dist = np.linalg.norm(vec1 - vec2)
dist

38870.68146369979